In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas
import datetime
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import sqlite3
import create_onset_data

In [2]:
inds_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
sm_db_name = "smu_sml_sme.sqlite"
sm_table_name = "smusmlsme"

In [3]:
# dataObj = create_onset_data.OnsetData(delTCutoff=2)
# ssBinDF = dataObj.create_output_bins(\
#                 saveFile="../data/binned_data_extra_latlon.feather",\
#                 aulDBdir=inds_dbdir, \
#                  aulDBName=sm_db_name,\
#                  aulTabName=sm_table_name)
# # print ssBinDF.head()
# # print "-------------"
# # print ssBinDF.shape

In [4]:
# ssBinDF.tail(6000)["data_label"].value_counts()

In [5]:
dataObj = create_onset_data.OnsetData(useSML=True,delTCutoff=2,\
                    smlDateRange=[datetime.datetime(1997,1,1),\
                    datetime.datetime(2000,1,1)], fillTimeRes=30,\
                    binTimeRes=60, nBins=1, dwnSmplByUT=True )
ssBinDF = dataObj.create_sml_bins()
print ssBinDF.head()
print "-------------"
print ssBinDF.shape

('Working through year-->', 1997)
('Working through year-->', 1998)
('Working through year-->', 1999)
('Working through year-->', 2000)
                     bin_0  mlat_0     mlt_0  del_minutes data_label
1997-01-01 00:00:00      0  -1.000 -1.000000         -1.0          S
1997-01-01 00:30:00      0  -1.000 -1.000000         -1.0          S
1997-01-01 01:00:00      0  -1.000 -1.000000         -1.0          S
1997-01-01 01:30:00      1   0.835  0.007306         46.0          S
1997-01-01 02:00:00      1   0.835  0.007306         16.0          S
('original DF label counts---->', 0    43672
1     8889
Name: outBinary, dtype: int64)
Downsampling the data into bins by UT hour
('new DF label counts---->', 1    8889
0    8889
Name: outBinary, dtype: int64)
                     bin_0  mlat_0     mlt_0  del_minutes data_label  \
1997-01-01 00:30:00      0  -1.000 -1.000000         -1.0          S   
1997-01-01 01:30:00      1   0.835  0.007306         46.0          S   
1997-01-01 02:00:00     

In [6]:
ssBinDF["hour"] = pandas.DatetimeIndex(ssBinDF.index).hour

test = ssBinDF.groupby(["hour", "outBinary"]).size()
print test

hour  outBinary
0     0            298
      1            298
1     0            291
      1            291
2     0            279
      1            279
3     0            332
      1            332
4     0            339
      1            339
5     0            365
      1            365
6     0            359
      1            359
7     0            413
      1            413
8     0            419
      1            419
9     0            417
      1            417
10    0            414
      1            414
11    0            403
      1            403
12    0            367
      1            367
13    0            365
      1            365
14    0            375
      1            375
15    0            393
      1            393
16    0            432
      1            432
17    0            432
      1            432
18    0            407
      1            407
19    0            394
      1            394
20    0            367
      1            367
21    0           